In [ ]:
#@title Mount Google Drive

from google.colab import drive
import glob

drive.mount('/content/gdrive')

input_folder = "/content/gdrive/My Drive/" #@param {type:"string"}
#@markdown - Folder containing query structure files
output_path = "/content/gdrive/My Drive/RESULTS.tsv" #@param {type:"string"}
#@markdown - Output path & name; results will be written into a .tsv file
#@markdown - Additional COMPLETE file will be written containing complete record of all hits
continuous_saving_mode = True #@param {type:"boolean"}
#@markdown - Appends results after each prediction

files = []
for file in glob.glob(f"/{input_folder.strip('/')}/*.pdb"):
  files.append(file)

print(f"Collected {len(files)} pdb files")

In [ ]:
#@title FoldSeek
from collections import Counter
import requests as rq
import time
import os
from IPython.display import Javascript

#limits the output window size
display(Javascript('google.colab.output.setIframeHeight(250)'))
#Prevents socket error
time.sleep(5)

#pLDDT calculator for AlphaFold models
def get_pLDDT_AlphaFold(file):
	residues_n = int()
	pLDDT_total = float()
	with open(file, "r") as f:
		current_residue = 0
		for l in f:

			if l.startswith("TER"):
				current_residue = 0

			if l.startswith("ATOM") and len(l.split()) == 12:
				if int(l.split()[5]) > current_residue:
					current_residue += 1
					residues_n += 1
					pLDDT_total += float(l.split()[-2])

			elif l.startswith("ATOM") and len(l.split()) == 11:
				try:
					if int(l.split()[4][1:]) > current_residue:
						current_residue += 1
						residues_n += 1
						pLDDT_total += float(l.split()[-2])

				except:
					if int(l.split()[5]) > current_residue:
						current_residue += 1
						residues_n += 1
						pLDDT_total += float(l.split()[-2])

	if residues_n == 0:
		return 0
	else:
		pLDDT_result = pLDDT_total/residues_n
		if pLDDT_result < 1:
			return pLDDT_result * 100
		else:
			return pLDDT_result

#pLDDT calculator for ESMFold structures
def get_pLDDT_ESMFold(file):
	residues_n = int()
	pLDDT_total = float()
	with open(file, "r") as f:
		current_residue = 0
		current_residue_atoms = int()
		current_residue_pLDDT = int()
		for l in f:

			if l.startswith("TER"):

				pLDDT_total += current_residue_pLDDT/current_residue_atoms

			if l.startswith("ATOM"):

				if int(l.split()[5]) > current_residue:
					residues_n += 1
					if current_residue_atoms != 0:
						pLDDT_total += current_residue_pLDDT/current_residue_atoms
					current_residue += 1
					current_residue_atoms = 1

					pLDDT_value = float(l.split()[-2])
					if pLDDT_value < 1.01:
						pLDDT_value = pLDDT_value * 100
					current_residue_pLDDT = pLDDT_value

				elif int(l.split()[5]) == current_residue:
					current_residue_atoms += 1
					pLDDT_value = float(l.split()[-2])
					if pLDDT_value < 1.01:
						pLDDT_value = pLDDT_value * 100
					current_residue_pLDDT += pLDDT_value

	if residues_n == 0:
		return 0
	else:
		return (pLDDT_total/residues_n)

#Calculates the pLDDT of a query structure if possible
def calculate_pLDDT(file):

  pLDDT = "No info"

  try:
    pLDDT = f"{get_pLDDT_AlphaFold(file):.2f}"
  except:
    pass

  if pLDDT == "No info":
    try:
      pLDDT = f"{get_pLDDT_ESMFold(file):.2f}"
    except:
      pass

  return pLDDT

#Atempts to retrieve UniProt annotation if available
def get_Uniprot_annotation(file):

  #Removes the prefix and sufix of files from AlphaFold DB
  if "AF-" in file and "-F1-model_v" in file:
    seq_ID = file.split("/")[-1][3:-16]
  else:
    seq_ID = file.split("/")[-1][:-4]

  fasta = rq.get(f"https://rest.uniprot.org/uniprotkb/{seq_ID}.fasta").text
  if fasta.startswith(">"):
    UniProt_annotation = " ".join(fasta.split()[1:])
    UniProt_annotation = UniProt_annotation[:UniProt_annotation.find("OS=")].strip()

    return UniProt_annotation

  #If  sequence ID hasn't been found in UniProtKB, returns 'Unknown'
  else:

    return "Unknown"

#Returns a list of annotations for FoldSeek hits
#Filteres out unannotated proteins and hits under the probability threshold
def get_annotation_list(file):

  probability_threshold = "0.500" #@param {type:"string"}
  #@markdown - Hits above threshold (0-1) will be included in the results
  #probability_threshold = "0.500"
  probability_threshold = float(probability_threshold)

  annotations = []
  annotations_IDs = []

  with open(file, "r") as f:
    for l in f:
      result_ID = l.split("\t")[1].split("-")[1]
      annotation = " ".join(l.split("\t")[1].split()[1:])
      probability = float(l.split("\t")[10])
      if not "uncharacterized" in annotation.lower() and not "hypothetical" in annotation.lower() and not "putative" in annotation.lower() and not "predicted" in annotation.lower() and probability >= probability_threshold:
        annotations.append(annotation)
        annotations_IDs.append(result_ID)

  return annotations, annotations_IDs

#Returns top FoldSeek hits for SwissProt, UniProt and AlphaFold_Proteome databases
#Returns most common hit annotation across all three databases and the number of occurences
def get_hits():

  DBs=["alis_afdb-swissprot.m8", "alis_afdb50.m8", "alis_afdb-proteome.m8"]
  annotations = []
  best_hits = {}
  complete_annotations = []

  #Retrieves best hits and creates list of all results
  for DB in DBs:
    new_DB, new_DB_IDs = get_annotation_list(DB)
    if len(new_DB) != 0:
      best_hits[DB] = new_DB[0]

      complete_DB_records = ""
      for i in range(len(new_DB)):
        complete_DB_records += new_DB_IDs[i] + " " + new_DB[i] + ";"

    else:
      best_hits[DB] = "None"
      complete_DB_records = "None"
    annotations.extend(new_DB)
    complete_annotations.append(complete_DB_records.strip(";"))

  #Retrieves the most common annotation among all hits
  most_common = Counter(annotations).most_common(1)
  keys_and_values = [(key, value) for key, value in most_common]
  if len(keys_and_values) != 0:
    most_common, number_of_hits = keys_and_values[0]
  else:
    most_common = "None"
    number_of_hits = 0

  #Appends all the results to a list
  results = []
  complete_results = []
  for DB in best_hits:
    results.append(best_hits[DB])
  results.append(most_common)
  results.append(str(number_of_hits))

  return results, complete_annotations

#Cleans the temporary files
def clean_tmp():
  os.system("rm /content/alis_*.m8")
  os.system("rm ticket.txt")
  os.system("rm Search_Output.gz")

#Uses the FoldSeek API and unzipps the results
#API call sometimes fail and produces corrupted .gz file!
def FoldSeek_call(file):

  print(f"{file} being predicted..")

  os.system(f"curl -X POST -F q=@{file} -F 'mode=3diaa' -F 'database[]=afdb50' -F 'database[]=afdb-swissprot' -F 'database[]=afdb-proteome' https://search.foldseek.com/api/ticket > /content/ticket.txt")

  #Retrieves unique FoldSeek run ID
  with open("/content/ticket.txt", "r") as f:
    try:
      ticket_id = f.read().split('"')[3]
    except IndexError:
      if "ticket_id" in locals():
        del ticket_id
      print(f"{file} prediction has failed.. ")

  if "ticket_id" in locals():
    #Retrieves FoldSeek results
    os.system(f"curl -X GET https://search.foldseek.com/api/result/download/{ticket_id} --output /content/Search_Output.gz")
    os.system(f'tar -xvzf "/content/Search_Output.gz"')

#Calls for the FoldSeek search, retrieves UniProt annotation and returns complete list of all annotation details
def FoldSeek(file):

  annotation = []

  #Gets sequence ID
  seq_ID = file.split("/")[-1][:-4]
  annotation.append(seq_ID)

  #Calculates pLDDT
  pLDDT = calculate_pLDDT(file)
  annotation.append(pLDDT)

  #Retrieves UniProt annotation if possible
  UniProt_annotation = get_Uniprot_annotation(file)
  annotation.append(UniProt_annotation)

  #API calls until *.m8 results are produced
  while True:
    if not os.path.exists("/content/alis_afdb-swissprot.m8") and not os.path.exists("/content/alis_afdb-proteome.m8") and not os.path.exists("/content/alis_afdb50.m8"):
      FoldSeek_call(file)
    else:
      break

  #Retrieves the FoldSeek annotations
  FoldSeek_results, complete_results = get_hits()
  annotation.extend(FoldSeek_results)

  clean_tmp()

  return annotation, complete_results

#Returns a string containing the results in TSV format
def make_table(rows_list):

  tsv = ""

  for row in rows_list:
    tsv += "\t".join(row) + "\n"

  return tsv

#Main
table_rows = [["Structure ID", "pLDDT","UniProt annotation", "Swiss-Prot", "UniProt", "AlphaFold-Proteomes","Most frequent hit", "Most frequent hit n"]]
complete_table_rows = [["Structure ID", "pLDDT", "UniProt annotations", "Swiss-Prot", "UniProt", "AlphaFold-Proteomes"]]

#Writes results after all the computations have been done
if continuous_saving_mode == False:
  for file in files:

    #Google Colab removes the white space
    file = file.replace("My Drive", "MyDrive")

    annotation, complete_annotation = FoldSeek(file)
    table_rows.append(annotation)
    complete_annotation.insert(0, annotation[2])
    complete_annotation.insert(0, annotation[1])
    complete_annotation.insert(0, annotation[0])
    complete_table_rows.append(complete_annotation)
    print(f"Prediction has been finished for {file}.. ")

  results_file = make_table(table_rows)
  complete_results_file = make_table(complete_table_rows)

  #Writes results into the output folder
  with open(f"/{output_path.strip('/')}", "w") as f:
    f.write(results_file)

  complete_results_path = output_path.strip("/").split(".")[0] + "_COMPLETE." + output_path.strip("/").split(".")[1]
  with open(f"/{complete_results_path}", "w") as f:
    f.write(complete_results_file)

#Continuously appends new rows to the result file
else:

  #Sets up the result files and writes the header
  with open(f"/{output_path.strip('/')}", "w") as f:
    f.write("\t".join(table_rows[0]) + "\n")

  complete_results_path = output_path.strip("/").split(".")[0] + "_COMPLETE." + output_path.strip("/").split(".")[1]
  with open(f"/{complete_results_path}", "w") as f:
    f.write("\t".join(complete_table_rows[0]) + "\n")
  for file in files:

    #Google Colab removes the white space
    file = file.replace("My Drive", "MyDrive")
    annotation, complete_annotation = FoldSeek(file)

    with open(f"/{output_path.strip('/')}", "a") as f:
      f.write("\t".join(annotation) + "\n")
    with open(f"/{complete_results_path}", "a") as f:
      f.write("\t".join(annotation[:3]) + "\t" + "\t".join(complete_annotation) + "\n")

    print(f"Prediction has been finished for {file}.. ")